In [ ]:
import numpy as np
import netCDF4 as nc
import pandas as pd
import xarray as xr
import matplotlib as mpl
import matplotlib.pyplot as plt
import urllib.request
import json
import os
import cartopy.crs as ccrs
import pytide
from dfm_models.project import FMmodel
from dfm_models.utils import observations as obs
from dfm_models.utils import analysis
from dfm_models.utils import visualization
from dfm_models.utils import calibration

from pathlib import Path

In [ ]:
import holoviews as hv
from holoviews import opts
hv.extension("bokeh")

from bokeh.io import curdoc, output_notebook
from bokeh.models import ColumnDataSource
from bokeh.plotting import figure, show
from bokeh.themes import built_in_themes

output_notebook()
curdoc().theme = "light_minimal"

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
# project files
d3d = Path("/mnt/c/Users/rdchlclj/Projects/MR_D3D_model/Delft3D")
data = Path("/mnt/e/MS_River_plume")
project = d3d / "tidal_constituent_boundary_conditions"
output = project / "output"
figures = project / "figures"

# NOAA COOPs
const_data = project / "constituent_data"
noaa_prediction_data = project / "noaa_COOPs_prediction_data"
tidal_stations_fn = Path(
    "/mnt/e/MS_River_plume/ArcPro/MyProject/output data/tidal_constituents_stations.xlsx"
)

tidal_stations_all = pd.read_excel(tidal_stations_fn, index_col=[0])

In [ ]:
# load model results
study = "prod_2016"

# Observations

In [ ]:
# clean up data
# tidal stations to drop
drop_stations = (
    "Pensacola",
    "East Fowl River Bridge",
    "Mobile State Docks",
    "Chickasaw Creek",
    "West Fowl River Bridge",
    "Bayou La Batre Bridge",
    "Grand Bay NERR, Mississippi Sound",
    "Pascagoula NOAA Lab",
    "Sabine Pass North",
    "Texas Point, Sabine Pass",
    "Weeks Bay, Mobile Bay",
)
tidal_stations = tidal_stations_all[~tidal_stations_all.name.isin(drop_stations)]

no_NAVD = [
    "Eugene_Island_North_of__Gulf_of_Mexico",
    "I-10_Bonnet_Carre_Floodway",
    "Port_Fourchon_Belle_Pass",
    "Grand_Isle",
    "Pilots_Station_East_S.W._Pass",
    "Pilottown",
    "Dog_River_Bridge",
]

In [ ]:
stations = {
    station_name.replace(" ", "_").replace(",", ""): station_id
    for _, (station_id, station_name) in tidal_stations[["id", "name"]].iterrows()
}
begin_date = "20160101"
end_date = "20161231"

Obs = obs.Observations()
Obs.load_COOPs_stations(stations)
Obs.gen_COOPs_harcons()
Obs.gen_COOPS_observed_water_levels(begin_date, end_date, "NAVD")
Obs.gen_COOPS_observed_water_levels(begin_date, end_date, "LMSL")

#Obs.gen_COOPS_predicted_water_levels(begin_date, end_date)
#Obs.gen_COOPS_predicted_water_levels(begin_date, end_date, "LMSL")

In [ ]:
case = "p02"
case_name = f"{case}_10sig_2016prod"
project_dir = data / f"Delft3d/models/{study}/{case_name}"
project_output = project_dir / "output"
his_fn = project_dir / f"{case}_merged_his.nc"

his_output = FMmodel.hisOutput(his_fn)
his_output.water_level_comp_data(list(stations.keys()))
results = FMmodel.Results(his_output)
prod = FMmodel.Model(case, project_dir, results=results)
prod.results.his_output.harmonic_analysis()

prod_amp_error, prod_phase_error = analysis.compute_harcon_error(Obs, prod.results)

# Model results

In [ ]:
obs_holomap = visualization.water_level_holomap(Obs.observed_water_levels_LMSL, "obs")

prod_holomap = visualization.water_level_holomap(
    prod.results.his_output.water_level_LMSL, case
)
prod_amp_holomap = visualization.harcon_error_holomap(
    prod_amp_error, case, drop_small=False
)
prod_phase_holomap = visualization.harcon_error_holomap(
    prod_phase_error, case, drop_small=False
)

In [ ]:
overlay = obs_holomap * prod_holomap
overlay = overlay.opts(
    opts.Curve(width=1400, height=450),
    opts.Curve("pred", color="blue"),
    opts.Curve(case, color="black"),
).opts(
    show_grid=True,
    #xlim=(pd.to_datetime("2018-01-15"), pd.to_datetime("2018-02-15")),
    ylim=(-0.75, 0.75),
)
overlay

In [ ]:
panel2 = (
    (prod_amp_holomap)
    .opts(
        opts.Bars(width=1400, height=450),
        opts.Bars("td_01", color="black"),
        opts.Bars("td05", color="red"),
    )
    .opts(show_grid=True)
)
layout = overlay + panel2
layout.cols(1)

In [ ]:
panel2 = (
    (td05_amp_holomap * td_01_amp_holomap)
    .opts(
        opts.Bars(width=1400, height=400),
        opts.Bars("td_01", color="black"),
        opts.Bars("td05", color="red"),
    )
    .opts(ylim=(-0.1, 0.1))
)
panel3 = (td_01_phase_holomap * td05_phase_holomap).opts(
    opts.Bars(width=1400, height=400),
    opts.Bars("td_01", color="black"),
    opts.Bars("td05", color="red"),
)

layout2 = panel2 + panel3
layout2.cols(1)

# calibration

In [ ]:
td_01_west_bndry = FMmodel.AstroWaterLevelBC(project_dir / "tide_west_ext.pli", project_dir / "WaterLevel.bc")
west_corr_factors = calibration.compute_amp_correction_factor(td_01_west_bndry, td01_amp_error, Obs.COOPs, thresh=0.005)
west_corr_fn = Path("west_astro_corr.bc")
calibration.write_correction_file(west_corr_fn, west_corr_factors)

In [ ]:
td_01_east_bndry = FMmodel.AstroWaterLevelBC(project_dir / "tide_east_ext.pli", project_dir / "WaterLevel.bc")
east_corr_factors = calibration.compute_amp_correction_factor(td_01_east_bndry, td01_amp_error, Obs.COOPs, thresh=0.005)
east_corr_fn = Path("east_astro_corr.bc")
calibration.write_correction_file(east_corr_fn, east_corr_factors)

In [ ]:
td_01_south_bndry = FMmodel.AstroWaterLevelBC(project_dir / "tide_south_ext.pli", project_dir / "WaterLevel.bc")
south_corr_factors = calibration.compute_amp_correction_factor(td_01_south_bndry, td01_amp_error, Obs.COOPs, thresh=0.005)
south_corr_fn = Path("south_astro_corr.bc")
calibration.write_correction_file(south_corr_fn, south_corr_factors)